## Comparison of batches 6 & 7: exploring heterozygosity

<br>
In batch 6 (haplotypes), I plotted heteryzogosity against read depth and ended up with several individuals with extremely high heterozygosity. 

For the same plot in batch 7 (random snps), I do not have any individuals with observed heterozygosities above 0.20. 

I think that this has something to do with using the haplotypes v. random snps method after `populations`, so I am going to... 
1. BLAST the batches 6 and 7 catalogs against each other to correlate loci names
2. Create a dictionary with batch 7 loci names as the key, and batch 6 loci names as the value. 
3. Choose a subset of individuals
4. Create a  list of loci that are missing genotypes in each individual
5. Find those loci in the batch 6 and see if they are also missing, called as heterozygotes, or called as homozygotes. 

#### 9/18/2017

### BLAST catalogs from batch 6 and batch 7

<br>
#### Create FASTA files from each catalog

In [1]:
cd ../analyses

/mnt/hgfs/Pacific cod/DataAnalysis/PCod-Korea-repo/analyses


In [2]:
cd batch6_v_batch7/

/mnt/hgfs/Pacific cod/DataAnalysis/PCod-Korea-repo/analyses/batch6_v_batch7


In [4]:
## Create FASTA from catalog.tags.tsv file: ##

infile = open("batch_6.catalog.tags.tsv", "r")
outfile = open("batch_6_catalog.fa", "w")

for line in infile:
    if not line.startswith("#"):
        linelist = line.strip().split()
        locus = linelist[3]
        seq = linelist[9]
        outfile.write(">" + locus + "\n" + seq + "\n")
infile.close()
outfile.close()

In [5]:
## Create FASTA from catalog.tags.tsv file: ##

infile = open("batch_7.catalog.tags.tsv", "r")
outfile = open("batch_7_catalog.fa", "w")

for line in infile:
    if not line.startswith("#"):
        linelist = line.strip().split()
        locus = linelist[3]
        seq = linelist[9]
        outfile.write(">" + locus + "\n" + seq + "\n")
infile.close()
outfile.close()

#### Create BLAST database out of batch 7 catalog fasta file

In [6]:
!makeblastdb -in batch_7_catalog.fa \
-parse_seqids \
-dbtype nucl \
-out batch_7_catalogDB



Building a new DB, current time: 09/18/2017 18:51:39
New DB name:   /mnt/hgfs/Pacific cod/DataAnalysis/PCod-Korea-repo/analyses/batch6_v_batch7/batch_7_catalogDB
New DB title:  batch_7_catalog.fa
Sequence type: Nucleotide
Keep Linkouts: T
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 25224 sequences in 1.20668 seconds.


In [9]:
!blastn -query batch_6_catalog.fa \
-db batch_7_catalogDB \
-out batch_6_BLASTto_batch_7_cat \
-outfmt 6

### Create dictionary for loci that match between batches 6 & 7

In [10]:
## create a list of reference loci that have only one alignment in batch 6 data ##
infile = open("batch_6_BLASTto_batch_7_cat", "r")
unique_loci = []
duplicates = 0
batch7to6_dict = {}

for line in infile:
    ref_locus = line.strip().split()[1]
    if ref_locus not in unique_loci:
        unique_loci.append(ref_locus)
        batch7to6_dict[ref_locus] = line.strip().split()[0]
    elif ref_locus in unique_loci:
        duplicates += 1
infile.close()
print len(unique_loci), " reference loci aligned to only one batch 6 locus"
print "there were ", duplicates, " duplicates"

24912  reference loci aligned to only one batch 6 locus
there were  28014  duplicates


In [12]:
outfile = open("batch7_unique_refloci.txt", "w")
for locus in batch7to6_dict.keys():
    outfile.write(locus + "\t" + batch7to6_dict[locus] + "\n")
outfile.close()

#### 9/19/2017

In [ ]:
# read back in loci for dictionary #
infile = open("batch7_unique_refloci.txt", "r")
batch7to6_dict = []
for line in infile:
    batch7to6_dict[line.strip().split()[0]] = line.strip().split()[1]
infile.close()

### Select a subset of individuals